In [39]:
from collections import Counter
import pandas as pd
import numpy

def create_vocab(texts : list):
    "Создание словаря из набора текстов"
    T = []
    for t in texts:
        T+=t
    vocab = list(set(T))
    return vocab

def p_word_bernoulli(w : str, texts : list):
    "Вычисление вероятности встретить слово word по корпусу texts в предположении Бернулли"
    inclusions = [int(w in t) for t in texts]
    P = sum(inclusions)/len(inclusions)
    return P

def P_word_multinomial(w : str, texts : list):
    "Вычисление вероятности встретить слово word по корпусу texts в предположении мультиномиальной модели"
    T = []
    for t in texts:
        T+=t
    P = T.count(word)/len(T)
    return P

def p_bernoulli_all(texts, vocab=None):
    "Вычисление вероятностей по Бернулли для каждого из слов в текстах"
    if vocab:
        v = vocab
    else:
        v = create_vocab(texts)
    ans = {w : p_word_bernoulli(w, texts) for w in v}
    return ans

def p_multinomial_all(texts, vocab=None):
    "Вычисление вероятностей в предположении мультиномиальной модели для каждого из слов в текстах"
    if vocab:
        v = vocab
    else:
        v = create_vocab(texts)
    
    T = []
    for t in texts:
        T+=t    
        
    cnt = Counter(T)
    cnt = dict(cnt)
    for k in cnt.keys():
        cnt[k]/=len(T)
    
    for w in set(vocab).difference(set(cnt.keys())):
        cnt[w] = 0
    return cnt
    

def PMI(manipulative_texts : list, non_manipulative_texts : list):
    "На вход: наборы ТОКЕНИЗИРОВАННЫХ текстов, на выход - датафрейм"
    all_texts = manipulative_texts + non_manipulative_texts
    v_man = create_vocab(manipulative_texts) 
    v_non_man = create_vocab(non_manipulative_texts)
    
    vocab = list(set(v_man + v_non_man))
    
    p_bern_man = p_bernoulli_all(manipulative_texts, vocab)
    p_bern_all = p_bernoulli_all(all_texts, vocab)
    p_mult_man = p_multinomial_all(manipulative_texts, vocab)
    p_mult_all = p_multinomial_all(all_texts, vocab)
    
    answer = pd.DataFrame(index=vocab, columns=['pmi_bern_nonlog','pmi_mult_nonlog', 'pmi_bern', 'pmi_mult'])
    for v in vocab:
        answer['pmi_bern_nonlog'][v] = float(p_bern_man[v]/p_bern_all[v])+1e-4
        answer['pmi_mult_nonlog'][v] = float(p_mult_man[v]/p_mult_all[v])+1e-4
        answer['pmi_bern'][v] = np.log(answer['pmi_bern_nonlog'][v])
        answer['pmi_mult'][v] = np.log(answer['pmi_mult_nonlog'][v])        
    return answer
    
    
    

In [40]:
man_text = ['Факела - вот горящее знаменье'.split(), 'Или знамя горящих сердец'.split(),
            'Что несут наше верное знание'.split(), 'Мы - начала начал мы - конец'.split(),
           'Мы сотрем и сожжем землю бренную'.split(), 'Мы найдем путь в другую вселенную'.split()]
non_man_text = ['Во всем мире свобода нетленная'.split(), 'Как таинственный отблеск углей'.split(),
                'Наступила мечта сокровенная'.split(), 'С откровенною волей своей'.split(), 'Темнота в мире ночь безответная'.split(),
               'Это просто другая вселенная'.split()]

In [41]:
a = PMI(man_text, non_man_text)

In [42]:
a

,pmi_bern_nonlog,pmi_mult_nonlog,pmi_bern,pmi_mult
С,0.0001,0.0001,-9.21034,-9.21034
волей,0.0001,0.0001,-9.21034,-9.21034
путь,2.0001,1.75768,0.693197,0.563992
мечта,0.0001,0.0001,-9.21034,-9.21034
конец,2.0001,1.75768,0.693197,0.563992
другую,2.0001,1.75768,0.693197,0.563992
сокровенная,0.0001,0.0001,-9.21034,-9.21034
отблеск,0.0001,0.0001,-9.21034,-9.21034
Мы,2.0001,1.75768,0.693197,0.563992
и,2.0001,1.75768,0.693197,0.563992
